In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import requests

import pickle

In [2]:
reference_path = "../../../results/road/freeflow/reference_api.parquet"
calibration_path = "../../../results/road/freeflow/calibration_cache_api.pickle"
output_path = "../../../results/road/freeflow/output.parquet"

routing_endpoint = "http://localhost:8054/router/road"

departure_time = 4 * 3600
maximum_batch_size = 400

In [3]:
# Load reference
df_reference = pd.read_parquet(reference_path)

In [4]:
# Prepare requests
df_reference["request_index"] = np.arange(len(df_reference))

# Convert to requests
request_list = []

for index, row in df_reference.iterrows():
    request_list.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": departure_time
    })

In [5]:
# Prepare querying
def query_requests(request_list, settings):
    df_response = []
    batch_index = 0

    while batch_index * maximum_batch_size < len(request_list):
        batch = request_list[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]

        response = requests.post("http://localhost:8054/router/road", json = {
            "batch": batch,
            "freespeed": settings
        })

        df_response.append(pd.DataFrame.from_records(response.json()))
        batch_index += 1

    return pd.concat(df_response)

In [6]:
# Obtain settings
with open(calibration_path, "rb") as f:
    history = pickle.load(f)

objective = np.inf
settings = None

for item in history:
    if item["objective"] < objective:
        settings = item["settings"]

In [7]:
df_response = query_requests(request_list, settings)

In [8]:
df_response = pd.merge(
    df_response, df_reference[["request_index", "trip_id"]], on = "request_index")

df_response = df_response.drop(
    columns = ["request_index"])

In [9]:
df_response.to_parquet(output_path)